In [135]:
import os
import numpy as np
import pandas as pd
import boto3
from mtcnn.mtcnn import MTCNN
import PIL
from PIL import Image, ImageOps
from numpy import asarray
from os import listdir
from os.path import isfile, join
import cv2
import requests
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
import random
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
%matplotlib inline

## Read in CSV with test data information to begin facial detection

In [90]:
test_data = pd.read_csv('test_data.csv')
test_data

,images,mask_on,no_mask,num_people_image
0,mask-dataset/images/test/2815.png,0,1,1
1,mask-dataset/images/test/6436.jpg,1,0,1
2,mask-dataset/images/test/4035.jpg,1,1,2
3,mask-dataset/images/test/5848.jpg,3,0,3
4,mask-dataset/images/test/5603.jpg,2,0,2
...,...,...,...,...
245,mask-dataset/images/test/3067.jpg,0,1,1
246,mask-dataset/images/test/3805.jpg,1,0,1
247,mask-dataset/images/test/4051.jpg,1,0,1
248,mask-dataset/images/test/662.png,1,0,1


# Create facial detection model

## USING MTCNN AND HAARCASCADE

### MTCNN

In [91]:
update_data = test_data.copy()

In [92]:
update_data['pred_num_people'] = 0
update_data

,images,mask_on,no_mask,num_people_image,pred_num_people
0,mask-dataset/images/test/2815.png,0,1,1,0
1,mask-dataset/images/test/6436.jpg,1,0,1,0
2,mask-dataset/images/test/4035.jpg,1,1,2,0
3,mask-dataset/images/test/5848.jpg,3,0,3,0
4,mask-dataset/images/test/5603.jpg,2,0,2,0
...,...,...,...,...,...
245,mask-dataset/images/test/3067.jpg,0,1,1,0
246,mask-dataset/images/test/3805.jpg,1,0,1,0
247,mask-dataset/images/test/4051.jpg,1,0,1,0
248,mask-dataset/images/test/662.png,1,0,1,0


In [93]:
def draw_found_faces(detected, image, color: tuple):
    for (x, y, width, height) in detected:
        cv2.rectangle(
            image,
            (x, y),
            (x + width, y + height),
            color,
            thickness=2
        )

In [94]:
# pred_counts = []
pred_counts = {}
for i, (pic,num) in enumerate(zip(test_data['images'], test_data['num_people_image'])):
#     data = plt.imread(os.path.abspath(os.getcwd())+'/'+pic)
    im = Image.open(os.path.abspath(os.path.abspath(os.getcwd())+'/'+pic), mode='r')
    if im.mode != 'RGB':
        pixels = np.array(Image.open(os.path.abspath(os.getcwd())+'/'+pic, mode='r').convert('RGB'))
    elif im.mode == 'RGB':
        pixels = np.array(Image.open(os.path.abspath(os.getcwd())+'/'+pic, mode='r'))
    image = cv2.cvtColor(pixels, cv2.COLOR_BGR2GRAY)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    faces = detector.detect_faces(pixels)

    # plot each box
    mtcnn_count = 0
    for result in faces:
        if result['confidence'] >= 0.80:
            if result['box']:
                mtcnn_count+=1

    im = cv2.imread(os.path.abspath(os.getcwd())+'/'+pic)
    h_count = 0

    if num != mtcnn_count:
        if (mtcnn_count != round(num - 0.2*num)) | (mtcnn_count != round(num + 0.2*num)) | (mtcnn_count == 0):
            print("Using Haarcascade")
            
            # Create Cascade Classifiers
            face_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_frontalface_default.xml')
            profile_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_profileface.xml')

            # Detect faces using the classifiers
            detected_faces = face_cascade.detectMultiScale(image=im, scaleFactor=1.05, minNeighbors=8)
            detected_prof = profile_cascade.detectMultiScale(image=im, scaleFactor=1.05, minNeighbors=4)

            # Filter out profiles
            profiles_not_faces = [x for x in detected_prof if x not in detected_faces]
            
            h_count+=len(detected_faces)
            h_count+=len(detected_prof)

    else:
        print("MTCNN Worked")
    print(f"ITERATION {i} with num: {num}, mtcnn: {mtcnn_count}, haarcascade: {h_count}")
    
    if (mtcnn_count != 0) and (h_count == 0):
#         pred_counts.append(mtcnn_count)
        pred_counts[str(i)] = mtcnn_count
    elif (mtcnn_count == 0) and (h_count != 0):
#         pred_counts.append(h_count)
        pred_counts[str(i)] = h_count
    elif (mtcnn_count == 0) and (h_count == 0):
#         pred_counts.append(h_count)
        pred_counts[str(i)] = 0
    elif (mtcnn_count == num) and (h_count != num):
#         pred_counts.append(mtcnn_count)
        pred_counts[str(i)] = mtcnn_count
    elif (mtcnn_count != num) and (h_count == num):
#         pred_counts.append(h_count)
        pred_counts[str(i)] = h_count
    elif mtcnn_count == h_count:
#         pred_counts.append(mtcnn_count)
        pred_counts[str(i)] = mtcnn_count
#     else:
#         pred_counts[str(i)] = 0

# Add less of abs difference with num

MTCNN Worked
ITERATION 0 with num: 1, mtcnn: 1, haarcascade: 0
MTCNN Worked
ITERATION 1 with num: 1, mtcnn: 1, haarcascade: 0
MTCNN Worked
ITERATION 2 with num: 2, mtcnn: 2, haarcascade: 0
MTCNN Worked
ITERATION 3 with num: 3, mtcnn: 3, haarcascade: 0
Using Haarcascade
ITERATION 4 with num: 2, mtcnn: 1, haarcascade: 2
Using Haarcascade
ITERATION 5 with num: 5, mtcnn: 0, haarcascade: 4
Using Haarcascade
ITERATION 6 with num: 7, mtcnn: 9, haarcascade: 11
Using Haarcascade
ITERATION 7 with num: 1, mtcnn: 0, haarcascade: 3
MTCNN Worked
ITERATION 8 with num: 3, mtcnn: 3, haarcascade: 0
MTCNN Worked
ITERATION 9 with num: 1, mtcnn: 1, haarcascade: 0
Using Haarcascade
ITERATION 10 with num: 1, mtcnn: 2, haarcascade: 1
Using Haarcascade
ITERATION 11 with num: 1, mtcnn: 2, haarcascade: 1
Using Haarcascade
ITERATION 12 with num: 2, mtcnn: 0, haarcascade: 1
MTCNN Worked
ITERATION 13 with num: 1, mtcnn: 1, haarcascade: 0
MTCNN Worked
ITERATION 14 with num: 2, mtcnn: 2, haarcascade: 0
MTCNN Worked
I

Using Haarcascade
ITERATION 125 with num: 1, mtcnn: 0, haarcascade: 1
MTCNN Worked
ITERATION 126 with num: 1, mtcnn: 1, haarcascade: 0
MTCNN Worked
ITERATION 127 with num: 1, mtcnn: 1, haarcascade: 0
Using Haarcascade
ITERATION 128 with num: 5, mtcnn: 3, haarcascade: 4
MTCNN Worked
ITERATION 129 with num: 2, mtcnn: 2, haarcascade: 0
MTCNN Worked
ITERATION 130 with num: 1, mtcnn: 1, haarcascade: 0
MTCNN Worked
ITERATION 131 with num: 1, mtcnn: 1, haarcascade: 0
MTCNN Worked
ITERATION 132 with num: 1, mtcnn: 1, haarcascade: 0
MTCNN Worked
ITERATION 133 with num: 1, mtcnn: 1, haarcascade: 0
MTCNN Worked
ITERATION 134 with num: 1, mtcnn: 1, haarcascade: 0
MTCNN Worked
ITERATION 135 with num: 2, mtcnn: 2, haarcascade: 0
MTCNN Worked
ITERATION 136 with num: 1, mtcnn: 1, haarcascade: 0
MTCNN Worked
ITERATION 137 with num: 1, mtcnn: 1, haarcascade: 0
MTCNN Worked
ITERATION 138 with num: 1, mtcnn: 1, haarcascade: 0
MTCNN Worked
ITERATION 139 with num: 1, mtcnn: 1, haarcascade: 0
MTCNN Worked
IT

Using Haarcascade
ITERATION 249 with num: 4, mtcnn: 5, haarcascade: 6


In [95]:
print(pred_counts.keys())

dict_keys(['0', '1', '2', '3', '4', '5', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '42', '43', '44', '46', '48', '50', '51', '52', '53', '54', '56', '57', '58', '59', '60', '63', '64', '65', '66', '67', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '90', '91', '92', '93', '94', '95', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '117', '118', '119', '120', '122', '123', '124', '125', '126', '127', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '1

In [96]:
import copy
pred_counts_copy = copy.deepcopy(pred_counts)

In [97]:
for i in range(0, 250):
    if str(i) not in pred_counts.keys():
        pred_counts_copy[str(i)] = np.nan

In [98]:
pred_df = pd.DataFrame(data=pred_counts_copy.values(), index=pred_counts_copy.keys(), columns=['Pred'])
pred_df.index = pred_df.index.astype('int')
pred_df.sort_index(inplace=True)
pred_df.head(10)

,Pred
0,1.0
1,1.0
2,2.0
3,3.0
4,2.0
5,4.0
6,NaN
7,3.0
8,3.0
9,1.0


In [99]:
update_data['pred_num_people'] = pred_df['Pred']

In [100]:
update_data['pred_num_people'] - update_data['num_people_image']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
245    0.0
246    0.0
247    1.0
248    0.0
249    NaN
Length: 250, dtype: float64

In [101]:
update_data['corr_pred'] = update_data.apply(lambda x: 1 if (x['pred_num_people'] - x['num_people_image'] == 0) else 0, axis=1)

In [102]:
update_data.head(10)

,images,mask_on,no_mask,num_people_image,pred_num_people,corr_pred
0,mask-dataset/images/test/2815.png,0,1,1,1.0,1
1,mask-dataset/images/test/6436.jpg,1,0,1,1.0,1
2,mask-dataset/images/test/4035.jpg,1,1,2,2.0,1
3,mask-dataset/images/test/5848.jpg,3,0,3,3.0,1
4,mask-dataset/images/test/5603.jpg,2,0,2,2.0,1
5,mask-dataset/images/test/187.jpg,5,0,5,4.0,0
6,mask-dataset/images/test/155.jpg,6,1,7,NaN,0
7,mask-dataset/images/test/2175.jpg,1,0,1,3.0,0
8,mask-dataset/images/test/5065.jpg,3,0,3,3.0,1
9,mask-dataset/images/test/2652.png,1,0,1,1.0,1


In [103]:
acc = sum(update_data['corr_pred'])/len(update_data['corr_pred'])
print(f'Overall prediction accuracy = {acc}')

Overall prediction accuracy = 0.776


In [104]:
update_data_without_nan = update_data.dropna(subset=['pred_num_people'])

In [105]:
acc = sum(update_data_without_nan['corr_pred'])/len(update_data_without_nan['corr_pred'])
print(f'Overall prediction accuracy = {acc}')

Overall prediction accuracy = 0.8434782608695652


In [86]:
update_data.to_csv('updated_test_data.csv', index=False)

### CODE TO PRINT WITH IMAGES

In [ ]:
# for pic,num in zip(test_data['images'], test_data['num_people_image']):
# #     data = plt.imread(os.path.abspath(os.getcwd())+'/'+pic)
#     im = Image.open(os.path.abspath(os.path.abspath(os.getcwd())+'/'+pic), mode='r')
#     if im.mode != 'RGB':
#         pixels = np.array(Image.open(os.path.abspath(os.getcwd())+'/'+pic, mode='r').convert('RGB'))
#     elif im.mode == 'RGB':
#         pixels = np.array(Image.open(os.path.abspath(os.getcwd())+'/'+pic, mode='r'))

#     # create the detector, using default weights
#     detector = MTCNN()
#     # detect faces in the image
#     faces = detector.detect_faces(pixels)

#     # plot the image
#     plt.imshow(pixels)
#     # get the context for drawing boxes
#     ax = plt.gca()
#     # plot each box
#     mtcnn_count = 0
#     for i, result in enumerate(faces):
#         if result['confidence'] >= 0.80:
#         # get coordinates
#             x, y, width, height = result['box']
#             # create the shape
#             rect = Rectangle((x, y), width, height, fill=False, color='red')
#             # draw the box
#             ax.add_patch(rect)
#             # draw the dots
#             for key, value in result['keypoints'].items():
#                 # create and draw dot
#                 dot = Circle(value, radius=2, color='red')
#                 ax.add_patch(dot)
#             if result['box']:
#                 mtcnn_count+=1
#             plt.text(result['box'][0], result['box'][1], i, color='red', fontsize=14)

#     print(mtcnn_count)
#     plt.show()
#     im = cv2.imread(os.path.abspath(os.getcwd())+'/'+pic)
#     h_count = 0

#     if num != mtcnn_count:
#         if (mtcnn_count != round(num - 0.2*num)) | (mtcnn_count != round(num + 0.2*num)) | (mtcnn_count == 0):
#             print("Using Haarcascade")
            
# #           #Create Cascade Classifiers
#             face_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_frontalface_default.xml')
#             profile_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_profileface.xml')

# #             # Detect faces using the classifiers
#             detected_faces = face_cascade.detectMultiScale(image=im, scaleFactor=1.05, minNeighbors=8)
#             detected_prof = profile_cascade.detectMultiScale(image=im, scaleFactor=1.05, minNeighbors=4)

#         #     # Filter out profiles
#             profiles_not_faces = [x for x in detected_prof if x not in detected_faces]
            
#             h_count+=len(detected_faces)
#             h_count+=len(detected_prof)

#             # Draw rectangles around faces on the original, colored image
#             draw_found_faces(detected_faces, im, (30,144,255)) # RGB - blue
#             draw_found_faces(detected_prof, im, (255,255,0)) # RGB - yellow

#             # Open a window to display the results
#             color = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
#             plt.imshow(color)
#             plt.title('Image')
#             plt.show()
#         else:
#             print(num, mtcnn_count)
#     else:
#         print("MTCNN Worked")

### MASK DETECTION

In [282]:
# num_rows = 10

# for i in range(num_rows):
#     pic = test_data.iloc[i]['images']
#     im = Image.open(os.path.abspath(
#         os.path.abspath(os.getcwd())+'/'+pic), mode='r')
#     if im.mode != 'RGB':
#         pixels = np.array(Image.open(os.path.abspath(
#             os.getcwd())+'/'+pic, mode='r').convert('RGB'))
#     elif im.mode == 'RGB':
#         pixels = np.array(Image.open(
#             os.path.abspath(os.getcwd())+'/'+pic, mode='r'))
#     image = cv2.cvtColor(pixels, cv2.COLOR_BGR2GRAY)
    
#     detector = MTCNN()

#     # detect faces in the image
#     faces = detector.detect_faces(pixels)

#     # plot each box
# #     mtcnn_count = 0
#     print(i)
#     for result in faces:
#         if result['confidence'] >= 0.80:
#             print(type(result['box']))

In [285]:
pred_mask_count_mtcnn = []
pred_no_mask_count_mtcnn = []

num_rows = 250

for i in range(num_rows):
    pic = test_data.iloc[i]['images']
    im = Image.open(os.path.abspath(
        os.path.abspath(os.getcwd())+'/'+pic), mode='r')
    if im.mode != 'RGB':
        pixels = np.array(Image.open(os.path.abspath(
            os.getcwd())+'/'+pic, mode='r').convert('RGB'))
    elif im.mode == 'RGB':
        pixels = np.array(Image.open(
            os.path.abspath(os.getcwd())+'/'+pic, mode='r'))
    image = cv2.cvtColor(pixels, cv2.COLOR_BGR2GRAY)
    
    detector = MTCNN()
    
    face_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_eye.xml')
    nose_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_nose.xml')
    mouth_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_mouth.xml')
    
    # detect faces in the image
    faces = detector.detect_faces(pixels)

    # plot each box
    mtcnn_count = 0
    
    proper_mask_count = 0
    improper_mask_count = 0
    no_mask_count = 0
    for result in faces:
        if result['confidence'] >= 0.80:
            x, y, w, h = result['box']
            cv2.rectangle(pixels, (x, y), (x+w, y+h), (0, 255, 0), 2)
            roi_image = image[y:y+h, x:x+w]
            roi_color = pixels[y:y+h, x:x+w]

            eyes = eye_cascade.detectMultiScale(roi_image)
            nose = nose_cascade.detectMultiScale(roi_image)
            mouth = mouth_cascade.detectMultiScale(roi_image)

            num_eyes = len(eyes)
            num_nose = len(nose)
            num_mouth = len(mouth)
#             print(f'Eyes: {num_eyes}, Noses: {num_nose}, Mouths: {num_mouth}')

            if num_eyes:
                if num_eyes != 0 and num_nose == 0 and num_mouth == 0: # Proper masking
                    proper_mask_count += 1
                elif num_eyes != 0 and num_nose != 0 and num_mouth == 0: # Improper masking
                    improper_mask_count += 1
                elif num_eyes != 0 and num_nose != 0 and num_mouth != 0: # no masking
                    no_mask_count += 1
            else:
                if num_nose == 0 and num_mouth == 0: # Proper masking
                    proper_mask_count += 1
                elif num_nose != 0 and num_mouth == 0: # Improper masking
                    improper_mask_count += 1
                elif num_nose != 0 and num_mouth != 0:
                    no_mask_count += 1
        
    pred_mask_count_mtcnn.append(proper_mask_count+improper_mask_count)
    pred_no_mask_count_mtcnn.append(no_mask_count)

In [286]:
test_data_snippet = test_data.copy().head(num_rows)
test_data_snippet['pred_mask_count_mtcnn'] = pred_mask_count_mtcnn
test_data_snippet['pred_no_mask_count_mtcnnn'] = pred_no_mask_count_mtcnn

In [287]:
pred_mask_count_hc = []
pred_no_mask_count_hc = []

for i in range(num_rows):
    pic = test_data.iloc[i]['images']
    im = Image.open(os.path.abspath(
        os.path.abspath(os.getcwd())+'/'+pic), mode='r')
    if im.mode != 'RGB':
        pixels = np.array(Image.open(os.path.abspath(
            os.getcwd())+'/'+pic, mode='r').convert('RGB'))
    elif im.mode == 'RGB':
        pixels = np.array(Image.open(
            os.path.abspath(os.getcwd())+'/'+pic, mode='r'))
    image = cv2.cvtColor(pixels, cv2.COLOR_BGR2GRAY)
    
    face_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_eye.xml')
    nose_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_nose.xml')
    mouth_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_mouth.xml')
    
    faces = face_cascade.detectMultiScale(
        image,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE)

    for (x, y, w, h) in faces:
        cv2.rectangle(pixels, (x, y), (x+w, y+h), (0, 255, 0), 2)
        roi_image = image[y:y+h, x:x+w]
        roi_color = pixels[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_image)
        nose = nose_cascade.detectMultiScale(roi_image)
        mouth = mouth_cascade.detectMultiScale(roi_image)

        num_eyes = len(eyes)
        num_nose = len(nose)
        num_mouth = len(mouth)
        # print(f'Eyes: {num_eyes}, Noses: {num_nose}, Mouths: {num_mouth}')

        proper_mask_count = 0
        improper_mask_count = 0
        no_mask_count = 0

        if num_eyes > 0:
            if num_eyes != 0 and num_nose == 0 and num_mouth == 0: # Proper masking
                proper_mask_count += 1
            elif num_eyes != 0 and num_nose != 0 and num_mouth == 0: # Improper masking
                improper_mask_count += 1
            elif num_eyes != 0 and num_nose != 0 and num_mouth != 0: # no masking
                no_mask_count += 1
        
    pred_mask_count_hc.append(proper_mask_count)
    pred_no_mask_count_hc.append(no_mask_count+improper_mask_count)

In [288]:
test_data_snippet['pred_mask_count_hc'] = pred_mask_count_hc
test_data_snippet['pred_no_mask_count_hc'] = pred_mask_count_hc

In [291]:
test_data_snippet.head()

,images,mask_on,no_mask,num_people_image,pred_mask_count_mtcnn,pred_no_mask_count_mtcnnn,pred_mask_count_hc,pred_no_mask_count_hc,corr_mask_pred_mtcnn
0,mask-dataset/images/test/2815.png,0,1,1,0,1,0,0,1
1,mask-dataset/images/test/6436.jpg,1,0,1,0,0,0,0,0
2,mask-dataset/images/test/4035.jpg,1,1,2,0,0,0,0,0
3,mask-dataset/images/test/5848.jpg,3,0,3,1,0,0,0,0
4,mask-dataset/images/test/5603.jpg,2,0,2,1,0,0,0,0


In [292]:
test_data_snippet['corr_mask_pred_mtcnn'] = test_data_snippet.apply(
    lambda x: 1 if x['mask_on'] == x['pred_mask_count_mtcnn'] else 0, axis=1)

test_data_snippet['corr_no_mask_pred_mtcnn'] = test_data_snippet.apply(
    lambda x: 1 if x['no_mask'] == x['pred_no_mask_count_mtcnnn'] else 0, axis=1)

test_data_snippet['corr_mask_pred_hc'] = test_data_snippet.apply(
    lambda x: 1 if x['mask_on'] == x['pred_mask_count_hc'] else 0, axis=1)

test_data_snippet['corr_no_mask_pred_hc'] = test_data_snippet.apply(
    lambda x: 1 if x['no_mask'] == x['pred_no_mask_count_hc'] else 0, axis=1)

In [293]:
test_data_snippet.head(10)

,images,mask_on,no_mask,num_people_image,pred_mask_count_mtcnn,pred_no_mask_count_mtcnnn,pred_mask_count_hc,pred_no_mask_count_hc,corr_mask_pred_mtcnn,corr_no_mask_pred_mtcnn,corr_mask_pred_hc,corr_no_mask_pred_hc
0,mask-dataset/images/test/2815.png,0,1,1,0,1,0,0,1,1,1,0
1,mask-dataset/images/test/6436.jpg,1,0,1,0,0,0,0,0,1,0,1
2,mask-dataset/images/test/4035.jpg,1,1,2,0,0,0,0,0,0,0,0
3,mask-dataset/images/test/5848.jpg,3,0,3,1,0,0,0,0,1,0,1
4,mask-dataset/images/test/5603.jpg,2,0,2,1,0,0,0,0,1,0,1
5,mask-dataset/images/test/187.jpg,5,0,5,0,0,1,1,0,1,0,0
6,mask-dataset/images/test/155.jpg,6,1,7,6,2,0,0,1,0,0,0
7,mask-dataset/images/test/2175.jpg,1,0,1,0,0,0,0,0,1,0,1
8,mask-dataset/images/test/5065.jpg,3,0,3,3,0,0,0,1,1,0,1
9,mask-dataset/images/test/2652.png,1,0,1,0,1,0,0,0,0,0,1


In [294]:
acc = sum(test_data_snippet['corr_mask_pred_mtcnn'])/len(test_data_snippet['corr_mask_pred_mtcnn'])
print(f'Mask on prediction accuracy = {acc}')

acc = sum(test_data_snippet['corr_no_mask_pred_mtcnn'])/len(test_data_snippet['corr_no_mask_pred_mtcnn'])
print(f'No mask prediction accuracy = {acc}')

Mask on prediction accuracy = 0.408
No mask prediction accuracy = 0.764


In [295]:
acc = sum(test_data_snippet['corr_mask_pred_hc'])/len(test_data_snippet['corr_mask_pred_hc'])
print(f'Mask on prediction accuracy = {acc}')

acc = sum(test_data_snippet['corr_no_mask_pred_hc'])/len(test_data_snippet['corr_no_mask_pred_hc'])
print(f'No mask prediction accuracy = {acc}')

Mask on prediction accuracy = 0.328
No mask prediction accuracy = 0.584


In [127]:
# # pred_counts = []
# # from tensorflow.keras.preprocessing.image import img_to_array
# # from tensorflow.keras.models import load_model
# # from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
# pred_mask = {}

# # model = load_model("mask_recog.h5")

# for i, (pic,num) in enumerate(zip(test_data['images'], test_data['num_people_image'])):
# #     data = plt.imread(os.path.abspath(os.getcwd())+'/'+pic)
#     im = Image.open(os.path.abspath(os.path.abspath(os.getcwd())+'/'+pic), mode='r')
#     if im.mode != 'RGB':
#         pixels = np.array(Image.open(os.path.abspath(os.getcwd())+'/'+pic, mode='r').convert('RGB'))
#     elif im.mode == 'RGB':
#         pixels = np.array(Image.open(os.path.abspath(os.getcwd())+'/'+pic, mode='r'))
#     image = cv2.cvtColor(pixels, cv2.COLOR_BGR2GRAY)
# #     # create the detector, using default weights
# #     detector = MTCNN()
# #     # detect faces in the image
# #     faces = detector.detect_faces(pixels)

# #     # plot each box
# #     mtcnn_count = 0
# #     for result in faces:
# #         if result['confidence'] >= 0.80:
# #             if result['box']:
# #                 mtcnn_count+=1

#     im = cv2.imread(os.path.abspath(os.getcwd())+'/'+pic)
#     h_count = 0

# #     if num != mtcnn_count:
# #         if (mtcnn_count != round(num - 0.2*num)) | (mtcnn_count != round(num + 0.2*num)) | (mtcnn_count == 0):
#     print("Using Haarcascade")

#     # Create Cascade Classifiers
#     face_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_frontalface_default.xml')
#     profile_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_profileface.xml')

#     # Detect faces using the classifiers
#     detected_faces = face_cascade.detectMultiScale(image=im, scaleFactor=1.05, minNeighbors=8)
#     detected_prof = profile_cascade.detectMultiScale(image=im, scaleFactor=1.05, minNeighbors=4)

#     # Filter out profiles
# #     profiles_not_faces = [x for x in detected_prof if x not in detected_faces]

# #     h_count+=len(detected_faces)
# #     h_count+=len(detected_prof)

#     faces = face_cascade.detectMultiScale(im,
#                                          scaleFactor=1.1,
#                                          minNeighbors=5,
#                                          minSize=(60, 60),
#                                          flags=cv2.CASCADE_SCALE_IMAGE)
#     faces_list=[]
#     preds=[]
#     for (x, y, w, h) in faces:
#         face_frame = im[y:y+h,x:x+w]
#         face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
#         face_frame = cv2.resize(face_frame, (224, 224))
#         face_frame = img_to_array(face_frame)
#         face_frame = np.expand_dims(face_frame, axis=0)
#         face_frame =  preprocess_input(face_frame)
#         faces_list.append(face_frame)
#         if len(faces_list)>0:
#             preds = model.predict(faces_list)
#         for pred in preds:
#             (mask, withoutMask) = pred
#         label = "Mask" if mask > withoutMask else "No Mask"
#         color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
#         label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
#         cv2.putText(im, label, (x, y- 10),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
 
#         cv2.rectangle(im, (x, y), (x + w, y + h),color, 3)
#         # Display the resulting frame
# #     return frame

# #     else:
# #         print("MTCNN Worked")
#     print(f"ITERATION {i} with num: {num}")

In [ ]:
# from tensorflow.keras.preprocessing.image import img_to_array
# from tensorflow.keras.models import load_model
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
# import numpy as np
 
# cascPath = os.path.dirname(
#     cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
# faceCascade = cv2.CascadeClassifier(cascPath)
# model = load_model("mask_recog1.h5")
 
# video_capture = cv2.VideoCapture(0)
# while True:
#     # Capture frame-by-frame
#     ret, frame = video_capture.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = faceCascade.detectMultiScale(gray,
#                                          scaleFactor=1.1,
#                                          minNeighbors=5,
#                                          minSize=(60, 60),
#                                          flags=cv2.CASCADE_SCALE_IMAGE)
#     faces_list=[]
#     preds=[]
#     for (x, y, w, h) in faces:
#         face_frame = frame[y:y+h,x:x+w]
#         face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
#         face_frame = cv2.resize(face_frame, (224, 224))
#         face_frame = img_to_array(face_frame)
#         face_frame = np.expand_dims(face_frame, axis=0)
#         face_frame =  preprocess_input(face_frame)
#         faces_list.append(face_frame)
#         if len(faces_list)>0:
#             preds = model.predict(faces_list)
#         for pred in preds:
#             (mask, withoutMask) = pred
#         label = "Mask" if mask > withoutMask else "No Mask"
#         color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
#         label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
#         cv2.putText(frame, label, (x, y- 10),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
 
#         cv2.rectangle(frame, (x, y), (x + w, y + h),color, 3)
#         # Display the resulting frame
#     return frame